# Install 

In [5]:
'''
download nnunet
'''
!nvidia-smi

# !git clone https://github.com/MIC-DKFZ/nnUNet.git
# !git clone https://github.com/Sungman-Cho/covid-challenge.git

Fri Mar 25 04:32:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 37%   49C    P0    51W / 280W |      0MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 86%   

In [8]:
# '''
# install nnunet
# '''

!pip install batchgenerators==0.21 medpy --user --quiet
!pip install -e . --quiet

# !pip install  pillow==7.0.0
#Run just 1 time, After run this, please reset your kernel as your workspace is in nnUNet

You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [9]:
# '''
# copy path file
# '''
!sudo cp paths.py nnunet/paths.py 

# genenrate pre-processing

In [13]:
import glob, natsort

############# provide task number ###########
num_task = 1
path_originalData = "nnUNet_raw_data_base/nnUNet_raw_data/Task08_Pancreas/"
# path_originalData = "nnUNet_raw_data_base/Task01_Covid/"
#################################################################


train_image = natsort.natsorted(glob.glob(path_originalData+"imagesTr/*.nii.gz"))
train_label = natsort.natsorted(glob.glob(path_originalData+"labelsTr/*.nii.gz"))
test_image = natsort.natsorted(glob.glob(path_originalData+"imagesTs/*.nii.gz"))
test_label = natsort.natsorted(glob.glob(path_originalData+"labelsTs/*.nii.gz"))

In [14]:
train_image = [
             "{}".format(patient_no.split('/')[-1]) for patient_no in train_image
]

train_label = [
             "{}".format(patient_no.split('/')[-1]) for patient_no in train_label
]

test_image = [
             "{}".format(patient_no.split('/')[-1]) for patient_no in test_image
]

len(train_image),len(train_label),len(test_image),len(test_label), train_image[0]

(4, 4, 1, 1, 'volume-covid19-A-0003_ct.nii.gz')

In [ ]:
import os
import json
from collections import OrderedDict

json_dict = OrderedDict()
json_dict['name'] = "Cardiac"
json_dict['description'] = "Combined (CT-MR) Healthy Abdominal Organ Segmentation"
json_dict['tensorImageSize'] = "3D"
json_dict['reference'] = "see challenge website"
json_dict['licence'] = "see challenge website"
json_dict['release'] = "0.0"
json_dict['modality'] = {
    "0": "CT"
}

#ch 4
json_dict['labels'] = {
    "0": "Background",
    "1": "Parenchyma",
    "2": "Abnormal",
}

json_dict['numTraining'] = len(train_image)
json_dict['numTest'] = len(test_image)

json_dict['training'] = []
for idx in range(len(train_image)):
    json_dict['training'].append({'image': "./imagesTr/%s" % train_image[idx], "label": "./labelsTr/%s" % train_label[idx]})

json_dict['test'] = ["./imagesTs/%s" % i for i in test_image]

with open(os.path.join(path_originalData, "dataset.json"), 'w') as f:
    json.dump(json_dict, f, indent=4, sort_keys=True)

In [ ]:
# !sudo nnUNet_convert_decathlon_task -i {path_originalData}
!python nnunet/experiment_planning/nnUNet_convert_decathlon_task.py -i {path_originalData}
print('done')

In [ ]:
# !nnUNet_plan_and_preprocess -t {num_task} #--verify_dataset_integrity
!python nnunet/experiment_planning/nnUNet_plan_and_preprocess.py -t {num_task}
print('done')

# Change Batch size in plans

In [ ]:
import pickle
#############################################################################
path_pickle = 'nnUNet_preprocessed/Task008_Pancreas/nnUNetPlansv2.1_plans_3D.pkl'
#############################################################################

with open(path_pickle, 'rb') as handle:
    temp = pickle.load(handle)    
    
print('Original stage 0 ',temp['plans_per_stage'][0]['batch_size'])
print('Original stage 1 ',temp['plans_per_stage'][1]['batch_size'])
#############################################################################
temp['plans_per_stage'][0]['batch_size'] = 2  # here is the batch size
temp['plans_per_stage'][1]['batch_size'] = 8  # here is the batch size
#############################################################################
print('After stage 0 ',temp['plans_per_stage'][0]['batch_size'])
print('After stage 1 ',temp['plans_per_stage'][1]['batch_size'])

import subprocess
subprocess.call(['/usr/bin/sudo', 'chmod', '777', path_pickle])

with open(path_pickle, 'wb') as handle:
    pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
#############################################################################
path_pickle = 'nnUNet_preprocessed/Task002_Tendon_4ch/nnUNetPlansv2.1_plans_3D.pkl'
#############################################################################

with open(path_pickle, 'rb') as handle:
    temp = pickle.load(handle)
    
    
print('Original stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# print('Original stage 1 ',temp['plans_per_stage'][1]['batch_size'])
#############################################################################
temp['plans_per_stage'][0]['batch_size'] = 8  # here is the batch size
# temp['plans_per_stage'][1]['batch_size'] = 8  # here is the batch size
#############################################################################
print('After stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# print('After stage 1 ',temp['plans_per_stage'][1]['batch_size'])


with open(path_pickle, 'wb') as handle:
    pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
#############################################################################
path_pickle = 'nnUNet_preprocessed/Task003_Tendon_5ch_new/nnUNetPlansv2.1_plans_3D.pkl'
#############################################################################

with open(path_pickle, 'rb') as handle:
    temp = pickle.load(handle)
    
    
print('Original stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# print('Original stage 1 ',temp['plans_per_stage'][1]['batch_size'])
#############################################################################
temp['plans_per_stage'][0]['batch_size'] = 8  # here is the batch size
# temp['plans_per_stage'][1]['batch_size'] = 8  # here is the batch size
#############################################################################
print('After stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# print('After stage 1 ',temp['plans_per_stage'][1]['batch_size'])


with open(path_pickle, 'wb') as handle:
    pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
#############################################################################
path_pickle = 'nnUNet_preprocessed/Task009_Covid/nnUNetPlansv2.1_plans_3D.pkl'
#############################################################################

with open(path_pickle, 'rb') as handle:
    temp = pickle.load(handle)
    
temp
# print('Original stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# # print('Original stage 1 ',temp['plans_per_stage'][1]['batch_size'])
# #############################################################################
# temp['plans_per_stage'][0]['batch_size'] = 8  # here is the batch size
# # temp['plans_per_stage'][1]['batch_size'] = 8  # here is the batch size
# #############################################################################
# print('After stage 0 :',temp['plans_per_stage'][0]['batch_size'])
# # print('After stage 1 ',temp['plans_per_stage'][1]['batch_size'])


# with open(path_pickle, 'wb') as handle:
#     pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
#############################################################################
path_pickle = 'OUTPUT_FOLDER/Task009/plans.pkl'
#############################################################################

with open(path_pickle, 'rb') as handle:
    temp = pickle.load(handle)
temp